In [2]:
import pandas as pd
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import datashader as ds
import colorcet as cc
import folium

from collections import defaultdict, OrderedDict
from folium.plugins import HeatMapWithTime
from datetime import datetime

In [3]:
files=os.listdir(r'C:\DAB_103_project\uber-pickups-in-new-york-city')[-7:]
files

['uber-raw-data-apr14.csv',
 'uber-raw-data-aug14.csv',
 'uber-raw-data-janjune-15.csv',
 'uber-raw-data-jul14.csv',
 'uber-raw-data-jun14.csv',
 'uber-raw-data-may14.csv',
 'uber-raw-data-sep14.csv']

In [ ]:
files.remove('uber-raw-data-janjune-15.csv')


In [ ]:
files

In [4]:
final=pd.DataFrame()
path=r'C:\DAB_103_project\uber-pickups-in-new-york-city'
for file in files:
    df=pd.read_csv(path+"/"+file,encoding='utf-8')
    final=pd.concat([df,final])

In [5]:
final.shape

(18804806, 8)

In [6]:
df=final.copy()

In [7]:
df.head()

,Date/Time,Lat,Lon,Base,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID
0,9/1/2014 0:01:00,40.2201,-74.0021,B02512,NaN,NaN,NaN,NaN
1,9/1/2014 0:01:00,40.7500,-74.0027,B02512,NaN,NaN,NaN,NaN
2,9/1/2014 0:03:00,40.7559,-73.9864,B02512,NaN,NaN,NaN,NaN
3,9/1/2014 0:06:00,40.7450,-73.9889,B02512,NaN,NaN,NaN,NaN
4,9/1/2014 0:11:00,40.8145,-73.9444,B02512,NaN,NaN,NaN,NaN


In [ ]:
df.tail()

In [ ]:
df.dtypes

In [ ]:
df['Date/Time'] = pd.to_datetime(df['Date/Time'], format="%m/%d/%Y %H:%M:%S")

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df['weekday']=df['Date/Time'].dt.day_name()
df['day']=df['Date/Time'].dt.day
df['minute']=df['Date/Time'].dt.minute
df['month']=df['Date/Time'].dt.month
df['hour']=df['Date/Time'].dt.hour

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df['Base'].unique()

In [ ]:
df['day'].unique()

In [ ]:
df['weekday'].unique()


In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
!pip install plotly

In [ ]:
plt.hist(df.weekday, bins=7, range =(-.5,6.5), rwidth=.8, color='#0041FD', alpha=.4,)
plt.xticks(range(7), 'Mon Tue Wed Thu Fri Sat Sun'.split())
plt.xlabel('Weekdays')
plt.ylabel('Total journay')
plt.title('Total journay by weekdays')
;

In [ ]:
df = df[
    df['Lon'].between(-74.25, -73.75) & 
    df['Lat'].between(40.5, 41)
]
print('Number of Pickups: {}'.format(len(df)))

pickup_points = ds.Canvas().points(df, 'Lon', 'Lat', )
ds.tf.set_background(ds.tf.shade(pickup_points, cmap=cc.b_rainbow_bgyr_35_85_c72), "black")

In [ ]:
df.head()

In [ ]:
uber_15=pd.read_csv(r'C:\DAB_103_project\uber-pickups-in-new-york-city\uber-raw-data-janjune-15.csv', encoding='utf-8')
uber_15.head()

In [ ]:
uber_15.info()

In [ ]:
uber_15.isnull().sum()

In [ ]:
#### remove null values

In [ ]:
uber=uber_15[uber_15['Affiliated_base_num'].notna()]

In [ ]:
uber.isnull().sum()

In [ ]:
uber['Pickup_date'].min()

In [ ]:
uber['Pickup_date'].max()

In [ ]:
uber_15['Pickup_date'] =  pd.to_datetime(uber_15['Pickup_date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
uber_15['weekday']=uber_15['Pickup_date'].dt.day_name()
uber_15['day']=uber_15['Pickup_date'].dt.day
uber_15['minute']=uber_15['Pickup_date'].dt.minute
uber_15['month']=uber_15['Pickup_date'].dt.month
uber_15['hour']=uber_15['Pickup_date'].dt.hour

In [ ]:
uber_15.head()

In [ ]:
df.Base.value_counts()

In [ ]:
base_names = {"Base": {'B02617':'Weiter', 'B02598':'Hinter','B02682':'Schmecken','B02764':'Danach-NY','B02512':'Unter'}}
df =df.copy()
df.replace(base_names, inplace=True)
df.head()

In [ ]:
import plotly.express as px
# Here we use a column of categorical data
fig = px.histogram(df, x="Base",
                  title='Pickups Per Base',
                   labels={'count':'No of Pickups'})
fig.show()

In [ ]:
uber_foil=pd.read_csv(r'C:\DAB_103_project\uber-pickups-in-new-york-city/Uber-Jan-Feb-FOIL.csv')

In [ ]:
uber_foil.head()

In [ ]:
uber_foil.info()

In [ ]:
uber_foil.isnull().sum()

In [ ]:
uber_foil['dispatching_base_number'].unique()

In [ ]:
sns.boxplot(x = 'dispatching_base_number', y = 'active_vehicles', data = uber_foil,)